In [1]:
import numpy as np
import pandas as pd
from sklearn import neighbors
from sklearn.cluster import KMeans

In [2]:
%%time
train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")

Wall time: 46.2 s


In [3]:
submit_flg=1 #全件処理かどうかをコントロール
grid_flg=1 #GridSearchをするかコントロール
SEED=12345
sample_num=10000
fold_num=5

#train関連
train_drop_col=['ID_code', 'target']
train_label='target'

#test関連
test_drop_col=['ID_code']

In [4]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
y_train=train_df[train_label]
x_test=test_df.drop(test_drop_col,axis=1)

特徴量作成コードを記載

In [ ]:
col_name='var_68'
df_var=train_df[col_name].value_counts().to_frame()
df_var=df_var.sort_index(ascending=True)
df_var=df_var.rename(columns={col_name: 'count'})
df_var

In [ ]:
pd.set_option('display.max_rows', 1000)
df_var[col_name]=0
df_var[col_name+'_areacat']=0
df_var['sum']=0

val_sum=0
threshold=2000
cat=0
for i in df_var.index:
    num=i
    count_val=df_var['count'].loc[i]
    if val_sum > threshold :
        val_sum=count_val
        cat+=1
    else:
        val_sum+=count_val
    df_var[col_name].loc[i]=num
    df_var[col_name+'_areacat'].loc[i]=cat
    df_var['sum'].loc[i]=val_sum
df_var

In [ ]:
train_df_var=train_df[['ID_code',col_name]]
train_df_var=pd.merge(train_df_var,df_var[[col_name,col_name+'_areacat']],how='left')
#train_df_var=train_df_var.sort_values('ID_code',ascending=True)
train_df[col_name+'_areacat']=train_df_var[col_name+'_areacat']
train_df

In [ ]:
x_train.columns

In [5]:
for j in ['var_68','var_91','var_12']:
#for j in x_train.columns:
    col_name=j
    print(j)
    df_var=train_df[col_name].value_counts().to_frame()
    df_var=df_var.sort_index(ascending=True)
    df_var=df_var.rename(columns={col_name: 'count'})
    
    df_var[col_name]=0
    df_var[col_name+'_areacat']=0
    df_var['sum']=0

    val_sum=0
    threshold=2000
    cat=0
    for i in df_var.index:
        num=i
        count_val=df_var['count'].loc[i]
        if val_sum > threshold :
            val_sum=count_val
            cat+=1
        else:
            val_sum+=count_val
        df_var[col_name].loc[i]=num
        df_var[col_name+'_areacat'].loc[i]=cat
        df_var['sum'].loc[i]=val_sum
    
    train_df_var=train_df[['ID_code',col_name]]
    train_df_var=pd.merge(train_df_var,df_var[[col_name,col_name+'_areacat']],how='left')
    train_df[col_name+'_areacat']=train_df_var[col_name+'_areacat']
    test_df_var=test_df[['ID_code',col_name]]
    test_df_var=pd.merge(test_df_var,df_var[[col_name,col_name+'_areacat']],how='left')
    test_df[col_name+'_areacat']=test_df_var[col_name+'_areacat']
train_df

var_68


C:\Users\tatsuya.okamoto\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


var_91
var_12


,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_68_areacat,var_91_areacat,var_12_areacat
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914,12,9,47
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518,55,27,49
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965,58,71,79
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996,55,60,17
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104,79,54,17
5,train_5,0,11.4763,-2.3182,12.6080,8.6264,10.9621,3.5609,4.5322,15.2255,...,0.4403,14.9452,1.0314,-3.6241,9.7670,12.5809,-4.7602,12,35,2
6,train_6,0,11.8091,-0.0832,9.3494,4.2916,11.1355,-8.0198,6.1961,12.0771,...,0.1777,18.3314,0.5845,9.1104,9.1143,10.8869,-3.2097,35,55,74
7,train_7,0,13.5580,-7.9881,13.8776,7.5985,8.6543,0.8310,5.6890,22.3262,...,5.8061,23.1407,-0.3776,4.2178,9.4237,8.6624,3.4806,18,13,91
8,train_8,0,16.1071,2.4426,13.9307,5.6327,8.8014,6.1630,4.4514,10.1854,...,1.4181,14.8370,-1.9940,-1.0733,8.1975,19.5114,4.8453,27,6,57
9,train_9,0,12.5088,1.9743,8.8960,5.4508,13.6043,-16.2859,6.0637,16.8410,...,3.6885,14.8344,0.4467,14.1287,7.9133,16.2375,14.2514,68,82,37


In [8]:
train_df.to_csv('./feature/107_train_areacat.csv',index=False)
test_df.to_csv('./feature/107_test_areacat.csv',index=False)